In [ ]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt 
from tqdm import tqdm 
#from watercalc import AFC_Wiener
import scipy as sc
import scipy.constants as scc
import statsmodels.api as sm
from scipy.optimize import curve_fit
import mdtraj as md
from tqdm import tqdm
kB = scc.Boltzmann

In [ ]:
def remove_water(path,file,save):
    
    
    struct = md.load(f"{path}" + f"/{file}")

    #traj = md.load("/home/nikolasmif98/Phe_Val/8Hydro/eql.trr",top=struct)

    selected = struct.atom_slice(struct.topology.select('resname PHE VAL')) #example for Phe-Val
    #selected_sol = struct.atom_slice(traj.topology.select('resname HOH'))

    print(selected)

    coordinates = selected.xyz

    coordinates.shape


    test = md.Trajectory(coordinates,topology=selected.topology).save(f'{path}' + f'/{save}')

In [ ]:
def get_peptide_traj(pth_gro,pth_traj,peptides,name_traj,name_gro):
    
    
    struct = md.load(pth_traj,top=pth_gro)


    selected = struct.atom_slice(struct.topology.select(f'resname {peptides}'))

    print(selected)

    coordinates = selected.xyz

    coordinates.shape


    test = md.Trajectory(coordinates,topology=selected.topology).save(f'/home/nikolasmif98/{name_traj}.xtc')

    
    
    struct = md.load(pth_gro)

    selected = struct.atom_slice(struct.topology.select(f'resname {peptides}'))

    print(selected)

    coordinates = selected.xyz

    coordinates.shape


    test = md.Trajectory(coordinates,topology=selected.topology).save(f'/home/nikolasmif98/{name_gro}.gro')

In [ ]:
def NN_pep(path,,top,file,save,res,res2,radius):
    
    """average Nearest neighbour calc

    path -- path to trajectories and gro fiels
    file -- name of file
    save -- save as
    top -- gro file (final frame)
    res1 and res2 -- residue of interest to extract 
    radius -- radius to be concidered as neighbours

   """

    
    remove_water(path,file,save) #Make gro file with no water
    
    #Now remove water from trajectory
    

    pth2 =  f'/home/nikolasmif98/10nm_Phe_Val_CG_2_2/FV_ionized/{file}.xtc'
    top =  f'/home/nikolasmif98/10nm_Phe_Val_CG_2_2/FV_ionized/{file}.gro'

    struct = md.load(pth2,top=top,stride=10)


    selected = struct.atom_slice(struct.topology.select(f'resname {res} {res2}'))
    #selected_sol = struct.atom_slice(traj.topology.select('resname HOH'))

    print(selected)

    coordinates = selected.xyz

    coordinates.shape


    test = md.Trajectory(coordinates,topology=selected.topology).save(f'/path/{save}.xtc')

    struct = md.load(f"/home/nikolasmif98/10nm_Phe_Val_CG_2_2/FV_ionized/{save}.gro")

    traj = md.load(f"/home/nikolasmif98/10nm_Phe_Val_CG_2_2/FV_ionized/{save}.xtc",top=struct)
    
    topo = traj.topology
    
    Neigh = []
    #liste = np.zeros(5001,)
    tr = np.zeros(1001)
    #tr = 0

    for i in tqdm(range(1,900,2)):

        ndx = topo.select(f'resid {i} {i+1}')#' {i+2}') #depending if dipeptide or higher order 
        #print(ndx)
        NN = np.array(md.compute_neighbors((traj),radius,ndx))

        tr = np.zeros(1001)

        for j in range(0,1001):
            k = len(NN[j])
            tr[j] = k
        Neigh.append(tr)
        
    avrg = np.array(Neigh)
    
    meen = np.mean(avrg,axis=0)#.shape

    err = np.std(avrg,axis=0)
    
    plt.plot(np.array(meen/7),'+--')